In [1]:
import pickle
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

2022-07-27 14:56:59.894283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 14:56:59.894344: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
with open('../data/keypoints.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

with open('../data/labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [8]:

le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [9]:
labels_num = le.transform(labels)
le.classes_.shape[0]

250

In [10]:
y = to_categorical(labels_num).astype(int)

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import os

2022-07-27 15:06:06.791636: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 15:06:06.791659: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True, activation='relu', input_shape=(None,126))))
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu')))
model.add(Bidirectional(LSTM(64, return_sequences=False, activation='relu')))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(le.classes_.shape[0], activation='softmax'))

In [77]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [73]:
keypoints[10].shape

(21, 126)

In [74]:
keypoints_pad = tf.keras.preprocessing.sequence.pad_sequences(keypoints, maxlen=30, dtype='float32',)

In [75]:
model.fit(keypoints_pad, y[:1597], epochs=250, callbacks=[tb_callback])

Epoch 1/250
50/50 [==============================] - 12s 129ms/step - loss: 5.5256 - categorical_accuracy: 0.0025
Epoch 2/250
50/50 [==============================] - 6s 130ms/step - loss: 5.5236 - categorical_accuracy: 0.0050
Epoch 3/250
50/50 [==============================] - 7s 133ms/step - loss: 5.7220 - categorical_accuracy: 0.0106
Epoch 4/250
50/50 [==============================] - 7s 134ms/step - loss: 5.7591 - categorical_accuracy: 0.0031
Epoch 5/250
50/50 [==============================] - 8s 151ms/step - loss: 8.2455 - categorical_accuracy: 0.0056
Epoch 6/250
50/50 [==============================] - 7s 149ms/step - loss: 6.7282 - categorical_accuracy: 0.0069
Epoch 7/250
33/50 [==================>...........] - ETA: 2s - loss: 5.6518 - categorical_accuracy: 0.0114

KeyboardInterrupt: 

In [34]:
# For webcam input:
path = 'ASL-Text/videos/ACCEPT/id_1178_Liz/id_1178_Liz_ACCEPT.avi'
cap = cv2.VideoCapture(0)
keypoints_frames = []
count = 0
result_label = ""
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
  while cap.isOpened():
    success, image = cap.read()
    
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    count += 1
    if results.multi_hand_landmarks:
      key_points = extract_keypoints(results)
      keypoints_frames.append(key_points)
      if count >=30:
        predict = model.predict(np.array([keypoints_frames]))[0]
        max_label = np.argmax(predict)
        if predict[max_label] > 0.5:
            result_label = le.inverse_transform([max_label])
        else:
            result_label = ""
        print(result_label)
        count = 0
        keypoints_frames = []
    
      
      for hand_landmarks in results.multi_hand_landmarks:
        #print('hand_landmarks:', hand_landmarks)
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        #print(hand_landmarks)
        #print(mp_hands.HAND_CONNECTIONS)
        
    # Flip the image horizontally for a selfie-view display.
    image = cv2.flip(image, 1)
    cv2.putText(image, ' '.join(result_label), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('MediaPipe Hands', image)
    
    if cv2.waitKey(5) & 0xFF == 27:
      break
    
cap.release()

1/1 [==============================] - 0s 13ms/step

1/1 [==============================] - 0s 14ms/step
['CONFLICT_OR_INTERSECTION']
1/1 [==============================] - 0s 16ms/step
['FLAT-TIRE']
1/1 [==============================] - 0s 14ms/step
['CONFLICT_OR_INTERSECTION']
1/1 [==============================] - 0s 13ms/step

1/1 [==============================] - 0s 15ms/step

1/1 [==============================] - 0s 16ms/step
['EXCUSE']
1/1 [==============================] - 0s 16ms/step
['CONFLICT_OR_INTERSECTION']
1/1 [==============================] - 0s 15ms/step
['OVER_OR_AFTER']
1/1 [==============================] - 0s 15ms/step
['CONFLICT_OR_INTERSECTION']
1/1 [==============================] - 0s 15ms/step
['CONFLICT_OR_INTERSECTION']
1/1 [==============================] - 0s 16ms/step
['NICE_OR_CLEAN']
1/1 [==============================] - 0s 16ms/step
['EXCUSE']
1/1 [==============================] - 0s 14ms/step
['CONFLICT_OR_INTERSECTION']
1/1 [==================

KeyboardInterrupt: 